In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../../')
import src.results as results
import pandas as pd

d:\ProgramData\Anaconda3\envs\rwn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def agg_res(res):
    new_res = {'classification':{}, 'link_pred':{}}

    for agg, runs in res.items():
        new_res[agg] = {}
        classification = []
        link_pred = []
        for r in runs:
            classification.append(r[0][2])
            link_pred.append(float(r[0][1].numpy()))
        new_res['classification'][agg] = classification
        new_res['link_pred'][agg] = link_pred


    print('Classification Acc')
    df = pd.DataFrame(new_res['classification'])
    display(df.mean())
    print()
    display(df.std())

    print('='*100)

    print('Link Pred AUC')
    df = pd.DataFrame(new_res['link_pred'])
    display(df.mean())
    print()
    display(df.std())

    return new_res


In [4]:
res = results.run_joint_train('Cora', 3, epochs=1000)
agg_res(res)


100%|██████████| 3/3 [4:01:54<00:00, 4838.12s/it]  

Classification Acc


eve     0.585949
mean    0.814225
gcn     0.896742
pool    0.810440
dtype: float64

eve     0.130415
mean    0.007494
gcn     0.001611
pool    0.006169
dtype: float64

Link Pred AUC


eve     0.270000
mean    0.649333
gcn     0.701333
pool    0.470333
dtype: float64

eve     0.260617
mean    0.044636
gcn     0.032130
pool    0.061134
dtype: float64

{'classification': {'eve': [0.45658138855820846,
   0.7173864019226882,
   0.5838777206262213],
  'mean': [0.8227470182610453, 0.8086655735495609, 0.8112612025785585],
  'gcn': [0.8958495990656096, 0.8986015588149412, 0.895775925967521],
  'pool': [0.8033170863188158, 0.8139502706587902, 0.8140526942341817]},
 'link_pred': {'eve': [0.09099999815225601,
   0.5690000057220459,
   0.15000000596046448],
  'mean': [0.6710000038146973, 0.5979999899864197, 0.6790000200271606],
  'gcn': [0.7260000109672546, 0.6650000214576721, 0.7129999995231628],
  'pool': [0.5289999842643738, 0.4749999940395355, 0.40700000524520874]},
 'eve': {},
 'mean': {},
 'gcn': {},
 'pool': {}}

In [4]:
# Do it with pubmed

res = results.run_joint_train('PubMed', 3, epochs=1000)
agg_res(res)

Using existing file ind.pubmed.x
Processing...
Done!
100%|██████████| 3/3 [4:02:23<00:00, 4847.98s/it]  

Classification Acc


eve     0.737060
mean    0.898413
gcn     0.893143
pool    0.882820
dtype: float64

eve     0.053358
mean    0.003586
gcn     0.000245
pool    0.006368
dtype: float64

Link Pred AUC


eve     0.698000
mean    0.782667
gcn     0.762333
pool    0.706667
dtype: float64

eve     0.045924
mean    0.008386
gcn     0.001528
pool    0.017954
dtype: float64

{'classification': {'eve': [0.7518711349359434,
   0.7814486305393821,
   0.6778612228692052],
  'mean': [0.8942732714834841, 0.900578519957415, 0.9003866285889299],
  'gcn': [0.8932078015645974, 0.8933495405196372, 0.8928729621358122],
  'pool': [0.876049974280422, 0.8837204425946351, 0.8886898652135274]},
 'link_pred': {'eve': [0.7260000109672546,
   0.6449999809265137,
   0.7229999899864197],
  'mean': [0.7870000004768372, 0.7730000019073486, 0.7879999876022339],
  'gcn': [0.7639999985694885, 0.7609999775886536, 0.7620000243186951],
  'pool': [0.6930000185966492, 0.7269999980926514, 0.699999988079071]},
 'eve': {},
 'mean': {},
 'gcn': {},
 'pool': {}}

In [4]:
res = results.run_joint_train('CiteSeer', 3, epochs=1000)
agg_res(res)

100%|██████████| 3/3 [17:42<00:00, 354.15s/it]

Classification Acc


mean    0.825211
gcn     0.859321
dtype: float64

mean    0.013721
gcn     0.019146
dtype: float64

Link Pred AUC


mean    0.550000
gcn     0.557333
dtype: float64

mean    0.048446
gcn     0.043662
dtype: float64

{'classification': {'mean': [0.8130249969810408,
   0.8225347180292236,
   0.8400724550175099],
  'gcn': [0.8664859316507669, 0.8376258905929236, 0.8738521917642798]},
 'link_pred': {'mean': [0.5920000076293945,
   0.5609999895095825,
   0.4970000088214874],
  'gcn': [0.6069999933242798, 0.5249999761581421, 0.5400000214576721]},
 'mean': {},
 'gcn': {}}

In [5]:
res = results.run_joint_train('CiteSeer', 3, epochs=1000)
agg_res(res)

100%|██████████| 3/3 [39:10:08<00:00, 47003.00s/it]   

Classification Acc


eve    0.538279
dtype: float64

eve    0.043107
dtype: float64

Link Pred AUC


eve    0.157667
dtype: float64

eve    0.057735
dtype: float64

{'classification': {'eve': [0.49997282936843374,
   0.5849583383649316,
   0.529905808477237]},
 'link_pred': {'eve': [0.19099999964237213,
   0.19099999964237213,
   0.09099999815225601]},
 'eve': {}}

In [4]:
res = results.run_joint_train('CiteSeer', 3, epochs=1000)
agg_res(res)

100%|██████████| 3/3 [38:38:32<00:00, 46370.81s/it]   

Classification Acc


pool    0.778197
dtype: float64

pool    0.006094
dtype: float64

Link Pred AUC


pool    0.403667
dtype: float64

pool    0.035501
dtype: float64

{'classification': {'pool': [0.7715879724670933,
   0.7835949764521193,
   0.7794082840236686]},
 'link_pred': {'pool': [0.37400001287460327,
   0.39399999380111694,
   0.4429999887943268]},
 'pool': {}}